# Создание разговорного приложения с RAG

Во многих вопросно-ответных приложениях пользователям нужна возможность вести разговор с сохранением контекста.
Для этого в приложении должна быть реализация памяти о прошлых вопросах и ответах, а также некоторая логика для использования их в текущих рассуждениях.

Информация в разделе посвящена добавлению логики для использования сообщений, полученных в течение разговора.
Подробнее об управлении историей чата — в разделе [Работа с историей сообщений](/docs/how_to/message_history).

В разделе рассмотрены два подхода:

1. Цепочки, в которых приложение всегда выполняет этап извлечения данных;
2. Агенты, в которых LLM самостоятельно принимает решение о выполнении этапа извлечения (или нескольких шагов) и способе его выполнения.

В качестве внешнего источника знаний использован тот же блог-пост [LLM Powered Autonomous Agents](https://lilianweng.github.io/posts/2023-06-23-agent/) Лилиан Венг из раздела [Создание RAG-приложения](/docs/tutorials/rag).

## Подготовка к работе

### Установка зависимостей

В этом руководстве используются эмбеддинги OpenAI и векторное хранилище Chroma, но все, что показано здесь, работает с любыми [моделями эмбеддингов](/docs/concepts#embedding-models), [векторными хранилищами](/docs/concepts#vectorstores) или [ретриверами](/docs/concepts#retrievers).

Вам понадобятся следующие пакеты:

In [1]:
%%capture --no-stderr
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-chroma bs4

Установите переменную окружения `OPENAI_API_KEY`.
Ее можно установить напрямую или загрузить из файла `.env`:

In [2]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass()

# import dotenv

# dotenv.load_dotenv()

<!--
### LangSmith

Многие приложения, которые вы создаете с LangChain, будут содержать несколько шагов с многократными вызовами LLM. По мере того как эти приложения становятся все более сложными, становится важно иметь возможность проверять, что именно происходит внутри вашей цепочки или агента. Лучший способ сделать это с помощью [LangSmith](https://smith.langchain.com).

Обратите внимание, что LangSmith не обязателен, но полезен. Если вы хотите использовать LangSmith, после регистрации по указанной выше ссылке, убедитесь, что установили переменные окружения для начала логирования трассировок:

```python
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()
```
-->

## Цепочки {#chains}

В первую очередь рассмотрим приложение, разработанное в разделе [Создание RAG-приложения](/docs/tutorials/rag).

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />
```

In [4]:
# | output: false
# | echo: false

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [6]:
import bs4
from langchain import hub
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 1. Загрузите, разделите на части и проиндексируйте содержимого блога, чтобы создать ретривер.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()


# 2. Включите ретривер в вопросно-ответную цепочку.
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [7]:
response = rag_chain.invoke({"input": "What is Task Decomposition?"})
response["answer"]

"Task decomposition involves breaking down complex tasks into smaller and simpler steps to make them more manageable for an agent or model. This process helps in guiding the agent through the various subgoals required to achieve the overall task efficiently. Different techniques like Chain of Thought and Tree of Thoughts can be used to decompose tasks into step-by-step processes, enhancing performance and understanding of the model's thinking process."

Обратите внимание, что в примере использованы встроенные конструкторы цепочек `create_stuff_documents_chain` и `create_retrieval_chain`.
Таким образом основными составляющими итогового решения являются:

1. Ретривер.
2. Промпт.
3. LLM.

Это упростит процесс включения истории разговора.

### Добавление истории разговора

Разработанная цепочка использует сам входной запрос для извлечения релевантного контекста.
Но в разговорной обстановке запрос пользователя может требовать дополнительного контекста беседы для понимания.
Например, представьте такой диалог:

> Human: "What is Task Decomposition?"
>
> AI: "Task decomposition involves breaking down complex tasks into smaller and simpler steps to make them more manageable for an agent or model."
>
> Human: "What are common ways of doing it?"

Для ответа на второй вопрос, система должна понимать, что "it" относится к "Task Decomposition".

Чтобы добиться этого в текущем приложении нужно изменить две вещи:

1. Доработать промпт, чтобы он мог использовать сообщения, полученные в течение разговора, в качестве входных данных.
2. Добавить подцепочку, которая берет последний вопрос пользователя и перефразирует его с учетом истории разговора. Создание такой цепочки можно рассматривать как разработку нового ретривера, который учитывает историю сообщений. То есть, если в первой итерации приложение работало так:
   - `запрос` -> `ретривер`
     
     То после доработки оно будет работать так:
   
   - `(запрос, история разговора)` -> `LLM` -> `переформулированный запрос` -> `ретривер`

#### Контекстуализация вопроса

Сначала вам нужно определить подцепочку, которая:

1. Берет сообщения, полученные в течение разговора, и последний вопрос пользователя.
2. Перефразирует вопрос, если он ссылается на какую-либо информацию в истории разговора.

Для этого используйте промпт, который включает переменную `MessagesPlaceholder` под именем `chat_history`.
Таким образом вы сможете передавать список сообщений в промпт, используя ключ `chat_history`.
Эти сообщения будут вставлены после системного сообщения и перед сообщением пользователя, содержащим последний вопрос.

Для решения этой задачи используйте вспомогательную функцию [create_history_aware_retriever](https://api.python.langchain.com/en/latest/chains/langchain.chains.history_aware_retriever.create_history_aware_retriever.html)
Она обрабатывает случаи, когда поле `chat_history` не содержит данных, и в противном случае применяет цепочку `prompt | llm | StrOutputParser() | retriever`.

Функция `create_history_aware_retriever` создает цепочку, которая принимает ключи `input` и `chat_history` в качестве входных данных и имеет такую же схему вывода, как и ретривер.

In [8]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

Эта цепочка добавляет перефразированный входной запрос к ретриверу, чтобы при извлечении данных приложение учитывало контекст беседы.

Теперь вы можете собрать итоговую вопросно-ответную цепочку.
Для этого достаточно заменить ретривер на новый `history_aware_retriever`.

Использовать функцию [create_stuff_documents_chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.combine_documents.stuff.create_stuff_documents_chain.html), чтобы создать цепочку`question_answer_chain` с входными ключами `context`, `chat_history` и `input`.
Эта цепочка принимает извлеченный контекст вместе с историей разговора и запросом для генерации ответа.

Создайте итоговую цепочку `rag_chain` с помощью функции [create_retrieval_chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.retrieval.create_retrieval_chain.html).
Эта цепочка последовательно применяет `history_aware_retriever` и `question_answer_chain`.
При этом цепочка сохраняет промежуточные выходные данные, такие как извлеченный контекст.
Она имеет входные ключи `input` и `chat_history`, и включает `input`, `chat_history`, `context` и `answer` в своем выводе.

In [9]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

Теперь попробуйте обратиться к приложению.
Задайте вопрос, ответ на который требует дополнительного контекста.
Поскольку цепочка включает входной параметр `"chat_history"`, историей разговора нужно управлять на стороне компонента, который вызывает цепочку.
Для этого вы можете добавлять входные и выходные сообщения в список:

In [10]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

question = "What is Task Decomposition?"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=ai_msg_1["answer"]),
    ]
)

second_question = "What are common ways of doing it?"
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])

Task decomposition can be achieved through various methods such as using techniques like Chain of Thought (CoT) or Tree of Thoughts to break down complex tasks into smaller steps. Common ways include prompting the model with simple instructions like "Steps for XYZ" or task-specific instructions like "Write a story outline." Human inputs can also be used to guide the task decomposition process effectively.


<!--
:::{.callout-tip}

Посмотрите [трассировку LangSmith](https://smith.langchain.com/public/243301e4-4cc5-4e52-a6e7-8cfe9208398d/r) 

:::
-->

#### Управление историей разговора с сохранением состояния

Вы уже знаете как добавить логику приложения для использования истории разговора, но пока ее нужно обновлять вручную и передавать при каждом вызове.
В реальном вопросно-ответном приложении вам понадобится как-то автоматизировать хранение, обновление и передачу истории разговора.

Для этого можно использовать:

- [BaseChatMessageHistory](https://api.python.langchain.com/en/latest/langchain_api_reference.html#module-langchain.memory). Класс, который хранит историю разговора.
- [RunnableWithMessageHistory](/docs/how_to/message_history). Класс-обертка для цепочки LCEL и `BaseChatMessageHistory`, которая обрабатывает передачу истории разговора на вход и обновляет ее после каждого вызова.

О том как использовать эти классы для создания цепочки диалога с сохранением состояния — в разделе [Работа с историей сообщений](/docs/how_to/message_history).

Ниже приведен пример решения, при котором история разговора хранится в простом словаре.
Для более надежного решения GigaChain предоставляет интеграции памяти с [Redis](/docs/integrations/memory/redis_chat_message_history/) и другими.

Экземпляры самостоятельно `RunnableWithMessageHistory` управляют историей разговора.
Они принимают конфигурацию с ключом (по умолчанию `"session_id"`), который указывает, что из истории разговора извлечь и добавить к входным данным, а что добавить к выходным данным той же истории разговоров.

Пример:

In [11]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [12]:
conversational_rag_chain.invoke(
    {"input": "What is Task Decomposition?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # constructs a key "abc123" in `store`.
)["answer"]

'Task decomposition involves breaking down complex tasks into smaller and simpler steps to make them more manageable. Techniques like Chain of Thought (CoT) and Tree of Thoughts help models decompose hard tasks into multiple manageable subtasks. This process allows agents to plan ahead and tackle intricate tasks effectively.'

In [13]:
conversational_rag_chain.invoke(
    {"input": "What are common ways of doing it?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

'Task decomposition can be achieved through various methods such as using Language Model (LLM) with simple prompting, task-specific instructions tailored to the specific task at hand, or incorporating human inputs to break down the task into smaller components. These approaches help in guiding agents to think step by step and decompose complex tasks into more manageable subgoals.'

Историю разговора можно просмотреть в словаре `store`:

In [14]:
for message in store["abc123"].messages:
    if isinstance(message, AIMessage):
        prefix = "AI"
    else:
        prefix = "User"

    print(f"{prefix}: {message.content}\n")

User: What is Task Decomposition?

AI: Task decomposition involves breaking down complex tasks into smaller and simpler steps to make them more manageable. Techniques like Chain of Thought (CoT) and Tree of Thoughts help models decompose hard tasks into multiple manageable subtasks. This process allows agents to plan ahead and tackle intricate tasks effectively.

User: What are common ways of doing it?

AI: Task decomposition can be achieved through various methods such as using Language Model (LLM) with simple prompting, task-specific instructions tailored to the specific task at hand, or incorporating human inputs to break down the task into smaller components. These approaches help in guiding agents to think step by step and decompose complex tasks into more manageable subgoals.



### Итоговое решение

Для удобства все шаги представлены в одной ячейке кода:

In [15]:
import bs4
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_chroma import Chroma
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)


### Создание ретривера ###
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()


### Контекстуализация вопроса ###
contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)


### Ответ на вопрос ###
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


### Управление историей разговора с сохранением состояния ###
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [16]:
conversational_rag_chain.invoke(
    {"input": "What is Task Decomposition?"},
    config={
        "configurable": {"session_id": "abc123"}
    },  # создает ключ "abc123" в `store`.
)["answer"]

'Task decomposition is a technique used to break down complex tasks into smaller and simpler steps. It involves transforming big tasks into multiple manageable tasks to facilitate problem-solving. Different methods like Chain of Thought and Tree of Thoughts can be employed to decompose tasks effectively.'

In [17]:
conversational_rag_chain.invoke(
    {"input": "What are common ways of doing it?"},
    config={"configurable": {"session_id": "abc123"}},
)["answer"]

'Task decomposition can be achieved through various methods such as using prompting techniques like "Steps for XYZ" or "What are the subgoals for achieving XYZ?", providing task-specific instructions like "Write a story outline," or incorporating human inputs to break down complex tasks into smaller components. These approaches help in organizing thoughts and planning ahead for successful task completion.'

## Агенты {#agents}

Агенты используют способность LLM к рассуждению для принятия решений в процессе выполнения.
Использование агентов позволяет делегировать часть процесса извлечения данных.
Хотя поведение агентов менее предсказуемо, чем у цепочек, они предлагают ряд преимуществ:

- агенты самостоятельно генерируют ввод для ретривера. То есть вам не нужно явно добавлять контекстуализацию, как показано в примере выше;
- Агенты могут выполнять несколько шагов извлечения данных для создания запроса или игнорировать этап извлечения, например, в ответ на общее приветствие пользователя.

### Инструмент для извлечения

Агенты могут использовать *инструменты* и управлять их выполнением.
В примере ниже показано как преобразовать ретривер в инструмент GigaChain, который будет использовать агент:

In [18]:
from langchain.tools.retriever import create_retriever_tool

tool = create_retriever_tool(
    retriever,
    "blog_post_retriever",
    "Searches and returns excerpts from the Autonomous Agents blog post.",
)
tools = [tool]

Инструменты являются экземплярами [Runnable](/docs/concepts#langchain-expression-language) и реализуют стандартный интерфейс:

In [19]:
tool.invoke("task decomposition")

'Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a classifier (via a prompt) or majority vote.\nTask decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.\n\nTree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first search) or DFS (depth-first search) with each state evaluated by a c

### Конструктор агента

После определения инструментов и LLM вы можете создать агента.
Для этого используйте [GigaGraph](/docs/concepts/#langgraph).
В представленных примерах используется высокоуровневый интерфейс для создания агента.
При этом преимущество GigaGraph в том, что высокоуровневый интерфейс поддерживается низкоуровневым API, который позволяет точно контролировать логику агента если нужно.

In [20]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, tools)

Попробуйте обратиться к агенту.

:::note

На данный момент агент не умеет работать с состояниями, вам все еще нужно добавить память.

:::

In [21]:
query = "What is Task Decomposition?"

for s in agent_executor.stream(
    {"messages": [HumanMessage(content=query)]},
):
    print(s)
    print("----")

Error in LangChainTracer.on_tool_end callback: TracerException("Found chain run at ID 1a50f4da-34a7-44af-8cbb-c67c90c9619e, but expected {'tool'} run.")


{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_1ZkTWsLYIlKZ1uMyIQGUuyJx', 'function': {'arguments': '{"query":"Task Decomposition"}', 'name': 'blog_post_retriever'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 68, 'total_tokens': 87}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-dddbe2d2-2355-4ca5-9961-1ceb39d78cf9-0', tool_calls=[{'name': 'blog_post_retriever', 'args': {'query': 'Task Decomposition'}, 'id': 'call_1ZkTWsLYIlKZ1uMyIQGUuyJx'}])]}}
----
{'tools': {'messages': [ToolMessage(content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on 

В GigaGraph есть встроенная поддержка сохранения состояния, поэтому вам не нужно использовать ChatMessageHistory. 
Вместо этого вы можете передать контроллер состояния напрямую своему агенту GigaGraph.

In [22]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")

agent_executor = create_react_agent(llm, tools, checkpointer=memory)

Этого достаточно для создания разговорного агента с RAG.

Посмотрите как будет работать агент.
Заметьте, что если вы введете запрос, который не требует этапа извлечения данных, агент его не выполнит:

In [23]:
config = {"configurable": {"thread_id": "abc123"}}

for s in agent_executor.stream(
    {"messages": [HumanMessage(content="Hi! I'm bob")]}, config=config
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='Hello Bob! How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 67, 'total_tokens': 78}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-022806f0-eb26-4c87-9132-ed2fcc6c21ea-0')]}}
----


Если вы введете запрос, который требует этапа извлечения данных, агент сгенерирует входные данные для вызова инструмента:

In [24]:
query = "What is Task Decomposition?"

for s in agent_executor.stream(
    {"messages": [HumanMessage(content=query)]}, config=config
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_DdAAJJgGIQOZQgKVE4duDyML', 'function': {'arguments': '{"query":"Task Decomposition"}', 'name': 'blog_post_retriever'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 91, 'total_tokens': 110}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-acc3c903-4f6f-48dd-8b36-f6f3b80d0856-0', tool_calls=[{'name': 'blog_post_retriever', 'args': {'query': 'Task Decomposition'}, 'id': 'call_DdAAJJgGIQOZQgKVE4duDyML'}])]}}
----


Error in LangChainTracer.on_tool_end callback: TracerException("Found chain run at ID 9a7ba580-ec91-412d-9649-1b5cbf5ae7bc, but expected {'tool'} run.")


{'tools': {'messages': [ToolMessage(content='Fig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. The model is instructed to “think step by step” to utilize more test-time computation to decompose hard tasks into smaller and simpler steps. CoT transforms big tasks into multiple manageable tasks and shed lights into an interpretation of the model’s thinking process.\n\nFig. 1. Overview of a LLM-powered autonomous agent system.\nComponent One: Planning#\nA complicated task usually involves many steps. An agent needs to know what they are and plan ahead.\nTask Decomposition#\nChain of thought (CoT; Wei et al. 2022) has become a standard prompting technique for enhancing model performance on complex tasks. 

Выше агент, вместо того чтобы вставлять запрос дословно в инструмент, удалил ненужные слова, такие как "what" и "is".

Этот же принцип позволяет агенту использовать контекст разговора, если нужно:

In [ ]:
query = "What according to the blog post are common ways of doing it? redo the search"

for s in agent_executor.stream(
    {"messages": [HumanMessage(content=query)]}, config=config
):
    print(s)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_KvoiamnLfGEzMeEMlV3u0TJ7', 'function': {'arguments': '{"query":"common ways of task decomposition"}', 'name': 'blog_post_retriever'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 930, 'total_tokens': 951}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3b956da36b', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-dd842071-6dbd-4b68-8657-892eaca58638-0', tool_calls=[{'name': 'blog_post_retriever', 'args': {'query': 'common ways of task decomposition'}, 'id': 'call_KvoiamnLfGEzMeEMlV3u0TJ7'}])]}}
----
{'action': {'messages': [ToolMessage(content='Tree of Thoughts (Yao et al. 2023) extends CoT by exploring multiple reasoning possibilities at each step. It first decomposes the problem into multiple thought steps and generates multiple thoughts per step, creating a tree structure. The search process can be BFS (breadth-first s

Заметьте, что агент смог понять, что "it" в запросе относится к "task decomposition", и сгенерировал обоснованный поисковый запрос — в данном случае, "common ways of task decomposition".

### Итоговое решение

Для удобства все шаги представлены в одной ячейке кода:

In [ ]:
import bs4
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.tools.retriever import create_retriever_tool
from langchain_chroma import Chroma
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.prebuilt import create_react_agent

memory = SqliteSaver.from_conn_string(":memory:")
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)


### Создание ретривера ###
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())
retriever = vectorstore.as_retriever()


### Создание инструмента ретривера ###
tool = create_retriever_tool(
    retriever,
    "blog_post_retriever",
    "Searches and returns excerpts from the Autonomous Agents blog post.",
)
tools = [tool]


agent_executor = create_react_agent(llm, tools, checkpointer=memory)

## Смотрите также

- Изучите различные типы ретриверов и методики извлечения в разделе [ретриверы](/docs/how_to/#retrievers).

- Ознакомьтесь с подробным руководствам по работе с разговорной памятью в GigaChain в разделе [Работа с историей сообщений](/docs/how_to/message_history).

- Ознакомьте подробнее с [агентами](/docs/tutorials/agents).